# Usage Analysis

In [1]:
import time
start_time = time.time()

## 1. Set an END date

In [2]:
end_date = '2023-11-12'

## 2. Add Link for Card Report

In [3]:
input_path = "J:/Reports BG/Card_Report_BG/ASPxGridViewCards_13.11.2023.csv"

## 4. Add Output Link 

In [4]:
output = "C:/Users/aleksandar.dimitrov/Desktop/PD/Usage_13.11.2023.xlsx"

## 3. Wait for the program... 

In [5]:
import pandas as pd
import pyodbc
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime

import win32com.client
import time

In [6]:
template_path = "C:/Users/aleksandar.dimitrov/Desktop/PD/Input/Usage_13.11.2023.xlsx" 

In [7]:
template = load_workbook(template_path)

In [8]:
worksheet = template.active

In [9]:
data = pd.read_csv(input_path, delimiter=";", encoding="cp1251")
data = data.drop(index=0)

C:\Users\aleksandar.dimitrov\AppData\Local\Temp\ipykernel_10640\4291810047.py:1: DtypeWarning: Columns (0,3,4,6,7,10,11,13,15,16,17,18,19,20,21,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(input_path, delimiter=";", encoding="cp1251")


In [10]:
data["Лимит"] = data["Лимит"].astype(float)
data["Дължимо до зануляване"] = data["Дължимо до зануляване"].astype(float)
data["Разполагаема сума"] = data["Лимит"] - data["Дължимо до зануляване"]

In [11]:
desired_products =  ["AXI 2", 
                    "AXI 2-500",
                    "Visa Free ATM World",
                    "Visa Бяла Карта - SC",
                    "Бяла Карта", 
                    "Бяла Карта – само лихва", 
                    "Бяла Карта 2",
                    "Бяла Карта 3", 
                    "Бяла Карта 3 - migrated",
                    "Бяла Карта 4.2%",
                    "Бяла Карта Gold - 10%",
                    "Бяла Карта Gold - 5%",                                         
                    "ИАМ"]

In [12]:
filtered_data_1 = data[
    (data["Текущ издател"] == "EPS") &
    (data["Продукт"].isin(desired_products)) &
    (data["Разполагаема сума"] > 70) & 
    (data["Статус на картата"] == "активна") &  
    (data["Дни забава"] == 0) ]   

In [13]:
filtered_data_2 = data[
    (data["Текущ издател"] == "EPS") &
    (data["Продукт"].isin(desired_products)) &
    (data["Разполагаема сума"] > 70) & 
    (data["Статус на картата"] == "активна") &  
    (data["Дни забава"] == "0")]   

In [14]:
columns_to_keep = ["Текущ издател",
          "Продукт",
          "EasyClientNumber",
          "Лимит",
          "Клиент",
          "ЕГН",
          "Телефон", 
          "Статус на картата",
          "Дължимо до зануляване",
          "Разполагаема сума",
          "Дни забава",
          "Дата на последно теглене на пари от картата"]

In [15]:
data_csv_1 = filtered_data_1[columns_to_keep]  
data_csv_1 = data_csv_1.rename(columns={"Дата на последно теглене на пари от картата": "Дата на последна транзакция"})
data_csv_1["Дата на последна транзакция"] = pd.to_datetime(data_csv_1["Дата на последна транзакция"]).dt.strftime("%d.%m.%Y")

In [16]:
data_csv_2 = filtered_data_2[columns_to_keep]  
data_csv_2 = data_csv_2.rename(columns={"Дата на последно теглене на пари от картата": "Дата на последна транзакция"})
data_csv_2["Дата на последна транзакция"] = pd.to_datetime(data_csv_2["Дата на последна транзакция"]).dt.strftime("%d.%m.%Y")

In [17]:
server = "scorpio"
database = "BIsmartWCBG"
conn = pyodbc.connect(f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes")

In [18]:
sql_query_1 = f"""

DECLARE @EndDate DATE = '{end_date}';

SELECT
EasyClientNumber,
ProcessorName,
FORMAT(MinActivatedDate, 'dd-MM-yyyy') AS Date,
Product
FROM(
SELECT DISTINCT EasyClientNumber,     
p.Name AS Product,
ProcessorName,
ROW_NUMBER() OVER(PARTITION BY EasyClientNumber  
                                ORDER BY ActivationDate asc) AS tr,
MIN(CONVERT(DATE,ActivationDate)) AS MinActivatedDate,
Limit
  FROM dwh.DimCards c
  left join dwh.DimOffers o On o.OfferSK = c.OfferSK
  left join dwh.DimProduct p ON p.ProductSK = o.ProductSK
  left join dwh.DimCardProcessors cp On cp.CardProcessorsSK = c.CardProcessorsSK
  where ActivationDate IS NOT NULL 
  group By EasyClientNumber,
Limit, 
p.Name,
ActivationDate,
ProcessorName
  ) as Result
  Where tr = 1 and 
  CONVERT(DATE,MinActivatedDate) between '2014-01-01' AND @EndDate  
  group By MinActivatedDate, 
  EasyClientNumber,
  Product,
  ProcessorName

"""

In [19]:
first_activation_date = pd.read_sql_query(sql_query_1, conn)

C:\Users\aleksandar.dimitrov\AppData\Local\Temp\ipykernel_10640\2463551554.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  first_activation_date = pd.read_sql_query(sql_query_1, conn)


In [20]:
first_activation_date = first_activation_date[["EasyClientNumber", "Date"]]

first_activation_date = first_activation_date.rename(columns=
                                                     {"Date": 
                                                      "Дата на първа активация на картата"})
first_activation_date["Дата на първа активация на картата"] = first_activation_date["Дата на първа активация на картата"].str.replace("-", ".")


In [21]:
sql_query_2 = """

SELECT DISTINCT 
    cards.EasyClientNumber,
    prod.Name,
    MIN(cards.ActivationDate) AS FirstActivationDate,
    offer.DateGenerateContract AS ContractDate
FROM dwh.DimCards AS cards
LEFT JOIN dwh.DimCardProcessors AS cp ON cp.CardProcessorID = cards.CardProcessorsSK
LEFT JOIN dwh.DimOffers AS offer ON offer.OfferSK = cards.OfferSK
LEFT JOIN dwh.DimProduct AS prod ON prod.ProductSK = offer.ProductSK
WHERE cards.CardProcessorsSK = 2
GROUP BY cards.EasyClientNumber, prod.Name, offer.DateGenerateContract
ORDER BY cards.EasyClientNumber;

"""

In [22]:
contract_date = pd.read_sql_query(sql_query_2, conn)

C:\Users\aleksandar.dimitrov\AppData\Local\Temp\ipykernel_10640\2744920968.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  contract_date = pd.read_sql_query(sql_query_2, conn)


In [23]:
contract_date = contract_date[["EasyClientNumber", "ContractDate"]]
contract_date = contract_date.rename(columns={"ContractDate": "Дата на подписване на договор"})
contract_date["Дата на подписване на договор"] = pd.to_datetime(contract_date["Дата на подписване на договор"])
contract_date["Дата на подписване на договор"] = contract_date["Дата на подписване на договор"].dt.strftime("%d.%m.%Y")

In [24]:
sql_query_3 = f"""

DECLARE @EndDate DATE = '{end_date}';
declare @DateConsentFrom date = convert(date, '2014-01-01');
declare @DateConsentTo date = convert(date, @EndDate);

WITH

--Emails --fix of duplicated emails
Emails AS (
SELECT ContactID, Email, Row_Number() OVER(PARTITION BY ContactID ORDER By Cdate desc) as rownum
FROM dwh.DimClientMails where [Current] = 1
)

SELECT  o.ContractNumber as EasyClientNumber, cl.EGN, 
CASE WHEN consent.HasMarketing= '1' THEN 'Yes' ELSE 'No' END AS Marketig_consent, 
CASE WHEN consent.HasEmail= '1' THEN 'Yes' ELSE 'No' END AS Email_Consent,
CASE WHEN consent.HasPhone= '1' THEN 'Yes' ELSE 'No' END AS HasPhone,
CASE WHEN consent.HasSMS= '1' THEN 'Yes' ELSE 'No' END AS SMS_Consent,
m.Email

FROM dwh.DimClientConsent as consent

JOIN dwh.DimOffers o ON o.OfferID = consent.OfferID
JOIN dwh.DimOffersStatus os ON os.OffStatusSK = o.OffStatusSK
JOIN dwh.DimProduct p ON p.ProductSK = o.ProductSK
JOIN dwh.DimClient cl ON cl.ClientSK = consent.ClientSK
--JOIN dwh.DimClientPhones ph ON ph.ContactID = consent.ContactOrPartnerId AND ph.[Current] = 1 AND ph.PhoneTypeSK = 1282
LEFT JOIN Emails m ON m.ContactID = consent.ContactOrPartnerId AND m.rownum = 1

WHERE consent.ClientSK <> -1 AND ContractNumber IS NOT NULL
AND CONVERT(DATE, consent.LDate) BETWEEN @DateConsentFrom AND @DateConsentTo

"""

In [25]:
consent_client = pd.read_sql_query(sql_query_3, conn)

C:\Users\aleksandar.dimitrov\AppData\Local\Temp\ipykernel_10640\3958564563.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  consent_client = pd.read_sql_query(sql_query_3, conn)


In [26]:
consent_client = consent_client[["EasyClientNumber",
                                 "Marketig_consent",
                                 "SMS_Consent",
                                 "Email_Consent",
                                 "Email"]]

In [27]:
data_csv_1 = data_csv_1.merge(first_activation_date, on="EasyClientNumber", how="left")
data_csv_1 = data_csv_1.merge(contract_date, on="EasyClientNumber", how="left")
data_csv_1 = data_csv_1.merge(consent_client, on="EasyClientNumber", how="left")

In [28]:
data_csv_2["EasyClientNumber"] = data_csv_2["EasyClientNumber"].astype(int)

In [29]:
data_csv_2 = data_csv_2.merge(first_activation_date, on="EasyClientNumber", how="left")
data_csv_2 = data_csv_2.merge(contract_date, on="EasyClientNumber", how="left")
data_csv_2 = data_csv_2.merge(consent_client, on="EasyClientNumber", how="left")

In [30]:
data_csv_1
data_csv_1["Продукт 2"] = data_csv_1["Продукт"]

data_csv_2
data_csv_2["Продукт 2"] = data_csv_2["Продукт"]


In [31]:
desired_order = ["Текущ издател",
          "Продукт",       
          "Продукт 2",
          "EasyClientNumber",
          "Лимит",
          "Клиент",
          "ЕГН",
          "Телефон", 
          "Статус на картата",
          "Дата на първа активация на картата",
          "Дължимо до зануляване",
          "Разполагаема сума",
          "Дни забава",
          "Marketig_consent",
          "SMS_Consent",
          "Email_Consent",
          "Email",
          "Дата на подписване на договор",
          "Дата на последна транзакция"]

In [32]:
data_csv_1 = data_csv_1[desired_order]

In [33]:
data_csv_2 = data_csv_2[desired_order]


In [34]:
data_result = pd.concat([data_csv_1, data_csv_2])

In [47]:
data_result.fillna("#N/A", inplace=True)
data_result

,Текущ издател,Продукт,Продукт 2,EasyClientNumber,Лимит,Клиент,ЕГН,Телефон,Статус на картата,Дата на първа активация на картата,Дължимо до зануляване,Разполагаема сума,Дни забава,Marketig_consent,SMS_Consent,Email_Consent,Email,Дата на подписване на договор,Дата на последна транзакция
0,EPS,Visa Бяла Карта - SC,Visa Бяла Карта - SC,470106,400.0,Иво Славейков Димитров,7207124024,359899543886,активна,14.06.2016,326.28,73.72,0,#N/A,#N/A,#N/A,#N/A,06.06.2016,26.09.2023
1,EPS,Бяла Карта 3 - migrated,Бяла Карта 3 - migrated,424881,900.0,Емил Жеков Иванов,7306093128,359878703980,активна,23.12.2014,-0.51,900.51,0,Yes,Yes,Yes,#N/A,08.12.2014,06.03.2017
2,EPS,Visa Бяла Карта - SC,Visa Бяла Карта - SC,838717,3200.0,Любка Иванова Бенчева,7103113537,359879202841,активна,07.08.2020,2320.31,879.69,0,Yes,Yes,Yes,LBENCHEVA11@ABV.BG,03.08.2020,12.11.2023
3,EPS,Бяла Карта 3 - migrated,Бяла Карта 3 - migrated,432230,900.0,Иван Спасов Георгиев,6303033041,359878375991,активна,08.06.2015,596.44,303.56,0,Yes,Yes,Yes,vatsche@abv.bg,01.06.2015,19.07.2023
4,EPS,Бяла Карта 3 - migrated,Бяла Карта 3 - migrated,475421,1200.0,Стефан Исталиянов Стефанов,6805271068,359878950800,активна,01.08.2016,0.00,1200.00,0,Yes,Yes,Yes,Stefisti@mail.bg,25.07.2016,05.07.2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5526,EPS,Бяла Карта 3 - migrated,Бяла Карта 3 - migrated,477223,1500.0,Петя Благоева Тодорова,6612043212,+359886028193,активна,16.08.2016,0.00,1500.00,0,Yes,No,Yes,ooopepi666@abv.bg,12.08.2016,23.05.2023
5527,EPS,Бяла Карта 3 - migrated,Бяла Карта 3 - migrated,426223,3000.0,Ивета Емилова Ангелова,8309166791,+359884390779,активна,14.01.2015,0.00,3000.00,0,Yes,Yes,Yes,iveta.angelova@abv.bg,12.01.2015,21.12.2021
5528,EPS,Бяла Карта 3 - migrated,Бяла Карта 3 - migrated,554288,800.0,Димитър Пенчев Колев,7008315429,+359899832961,активна,19.09.2018,0.00,800.00,0,Yes,Yes,Yes,MITKOO_DM@ABV.BG,17.09.2018,24.07.2023
5529,EPS,AXI 2,AXI 2,907263,2300.0,Десислава Павлова Големанска,7203222871,+359896730991,активна,16.03.2021,0.00,2300.00,0,Yes,Yes,Yes,desislavak@icloud.com,08.03.2021,27.10.2023


In [36]:
for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=worksheet.max_column):
    for cell in row:
        cell.value = None

In [37]:
data_rows = list(dataframe_to_rows(data_result, index=False, header=False))

In [38]:
start_row = 2
start_column = 1

In [39]:
for row in data_rows:
    for value in row:
        worksheet.cell(row=start_row, column=start_column, value=value)
        start_column += 1
    start_column = 1
    start_row += 1

In [40]:
template.save(output)

In [41]:
recipient_email = "alexi.zein@gmail.com"
subject = "Usage"
message = """Dear colleagues,

This email is automatically generated and contains information about added data.

Date and time of generation: {}

The Usage data has been added successfully and can be found in the shared folder on PD.

""".format(datetime.now().strftime("%d %B %Y, %H:%M ч."))

In [42]:
outlook = win32com.client.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")

In [43]:
mail = outlook.CreateItem(0)
mail.Subject = subject
mail.Body = message
mail.To = recipient_email

In [44]:
mail.Send()

In [45]:
print("The notification email has been successfully sent.")

The notification email has been successfully sent.


In [46]:
end_time = time.time()
execution_time_seconds = end_time - start_time
execution_time_minutes = execution_time_seconds / 60
print(f"The script executed for {execution_time_minutes:.2f} minutes.")

The script executed for 0.79 minutes.
